# Missing Values Exercises


## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_missing.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:


```python
assert set(results.keys()) == {
    "ex2_avg_income",
    "ex3_share_making_9999999",
    "ex3_share_making_zero",
    "ex5_avg_income",
    "ex8_avg_income_black",
    "ex8_avg_income_white",
    "ex8_racial_difference",
    "ex9_avg_income_black",
    "ex9_avg_income_white",
    "ex10_wage_gap",
}
```

### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.


## Exercises

### Exercise 1

Today, we will be using the ACS data we used during out first `pandas` exercise to examine the US income distribution, and how it varies by race. Note that because the US income distribution has a very small number of people with *extremely* high incomes, and the ACS is just a sample of Americans, the far right tail of the distribution will not be very well estimated. However, this data should suffice for helping to understand wealth inequality in the United States. 

To begin, load the ACS Data we used in our first pandas exercise. That [data can be found here](https://github.com/nickeubank/MIDS_Data/tree/master/US_AmericanCommunitySurvey). We'll be working with `US_ACS_2017_10pct_sample.dta`. 

In [1]:
# Importing libraries and loading data
import pandas as pd
import numpy as np

# seeting default option
pd.set_option("mode.copy_on_write", True)

df = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

# initialize results
results = {}

### Exercise 2

Let's begin by calculating the median US incomes from this data (recall that income is stored in the `inctot` variable). Store the answer in `results` under the key `"ex2_avg_income"`.

In [2]:
results["ex2_avg_income"] = df["inctot"].mean()

print(
    "The mean US income for 2017 as per the data is: ${:.2f}".format(
        results["ex2_avg_income"]
    )
)

The mean US income for 2017 as per the data is: $1723646.27


### Exercise 3

Hmmm... That doesn't look right. The average American is definitely not earning that much a year! Let's look at the values of `inctot` using `value_counts()`. Do you see a problem?

Now use `value_counts()` with the argument `normalize=True` to see proportions of the sample that report each value instead of the count of people in each category. What percentage of our sample has an income of 9,999,999? Store that proportion (between 0 and 1) as `"ex3_share_making_9999999"`. What percentage has an income of 0? Store that proportion as `"ex3_share_making_zero"`.

(Recall `.value_counts()` returns a Series, so you can pull values out with our usual pandas tools.)

In [3]:
df["inctot"].value_counts()

inctot
9999999    53901
0          33679
30000       4778
50000       4414
40000       4413
           ...  
70520          1
76680          1
57760          1
200310         1
505400         1
Name: count, Length: 8471, dtype: int64

In [4]:
df["inctot"].value_counts(normalize=True)

inctot
9999999    0.168967
0          0.105575
30000      0.014978
50000      0.013837
40000      0.013834
             ...   
70520      0.000003
76680      0.000003
57760      0.000003
200310     0.000003
505400     0.000003
Name: proportion, Length: 8471, dtype: float64

In [5]:
results["ex3_share_making_9999999"] = df["inctot"].value_counts(normalize=True)[9999999]
results["ex3_share_making_zero"] = df["inctot"].value_counts(normalize=True)[0]

print(
    "The proportion of people in the US that have an income of 9999999 is {:.2f}".format(
        results["ex3_share_making_9999999"]
    )
)
print(
    "The proportion of people in the US that have an income of 0 is {:.2f}".format(
        results["ex3_share_making_zero"]
    )
)

The proportion of people in the US that have an income of 9999999 is 0.17
The proportion of people in the US that have an income of 0 is 0.11


### Exercise 4

As we discussed before, the ACS uses a value of 9999999 to denote that income information is not available for someone. The problem with using this kind of "sentinel value" is that pandas doesn't understand that this is supposed to denote missing data, and so when it averages the variable, it doesn't know to ignore 9999999. 

To help out `pandas`, use the `replace` command to replace all values of 9999999 with `np.nan`. 

In [6]:
# Replacing sentinel value 9999999 with nan values

df1 = df.replace(9999999, np.nan, inplace=False)

### Exercise 5

Now that we've properly labeled our missing data as `np.nan`, let's calculate the average US income once more. Store the answer in `results` under the key `"ex5_avg_income"`.

In [7]:
results["ex5_avg_income"] = df1["inctot"].mean()
print(
    "The average income for US americans after the NA values were included instead of the sentinel values is {:.2f}".format(
        results["ex5_avg_income"]
    )
)

The average income for US americans after the NA values were included instead of the sentinel values is 40890.18


### Exercise 6

OK, now we've been able to get a reasonable average income number. As we can see, a major advantage of using `np.nan` is that `pandas` knows that `np.nan` observations should just be ignored when we are calculating means. 

But it's not enough to just get rid of the people who had `inctot` values of 9999999. We also need to know why those values were missing. Suppose, for example, that the value of 9999999 was used for anyone who made more than 100,000 dollars: if we just dropped those people, then our estimate of average income wouldn't mean much, would it?

So let's make sure we understand *why* data is missing for some people. If you recall from our last exercise, it seemed to be the case that most of the people who had incomes of 9999999 were children. Let's make sure that's true by looking at the distribution of the variable `age` for people for whom `inctot` is missing (i.e. subset the data to people with `inctot` missing, then look at the values of `age` with `value_counts()`).

Then do the opposite: look at the distribution of the `age` variable for people who whom `inctot` is *not* missing. 

Can you determine when 9999999 was being used? Is it ok we're excluding those people from our analysis?

Note: In this data, Python doesn't understand `age` is a number; it thinks it is a string because the original data has categories like "90 (90+ in 1980 and 1990)" and "less than 1 year old". So you can't just use `min()` or `max()`. We'll discuss converting string variables into numbers in a future class.

In [8]:
# counting the null values in inctot column specifying age
df1.loc[df1["inctot"].isnull(), "age"].value_counts()

age
10    3997
9     3977
14    3847
12    3845
13    3800
      ... 
39       0
38       0
37       0
36       0
96       0
Name: count, Length: 97, dtype: int64

In [9]:
# counting the value counts for the inctot column without null values
df1.loc[pd.notnull(df1.inctot)]["age"].value_counts()

age
60                      4950
54                      4821
59                      4776
56                      4776
58                      4734
                        ... 
5                          0
4                          0
3                          0
2                          0
less than 1 year old       0
Name: count, Length: 97, dtype: int64

### Exercise 7

Great, so now we know why those people had missing data, and we're ok with excluding them. 

But as we previously noted, there are also a lot of observations of zero income in our data, and it's not clear that we want everyone with a zero-income *should* be included in this average, since those may be people who are retired, or in school. 

Let's limit our attention to people who are currently working by subsetting to only employed respondents. We can do this using `empstat`. Remember you can use `value_counts()` to see what values of `empstat` are in the data!

In [10]:
# Creating subset dataframe
df2 = df1[df1["empstat"] == "employed"]

### Exercise 8

Now let's estimate the racial income gap in the United States. What is the average salary for employed Black Americans, and what is the average salary for employed White Americans? In percentage terms, how much more does the average White American make than the average Black American?

**Note:** these values are not quite accurate estimates. As we'll discuss in later lessons, to get completely accurate estimates from the ACS we have to take into account how people were selected to be interviewed. But you get pretty good estimates in most cases even without weights—your estimate of the racial wage gap without weights is within 5\% of the corrected value. 

**Note:** This is actually an underestimate of the wage gap. The US Census treats Hispanic respondents as a sub-category of "White." While all ethnic distinctions are socially constructed, and so on some level these distinctions are all deeply problematic, this coding is inconsistent with what most Americans think of when they hear the term "White," a term *most* Americans think of as a category that is mutually exclusive of being Hispanic or Latino (categories which are also usually conflated in American popular discussion). With that in mind, most researchers working with US Census data split "White" into "White, Hispanic" and "White, Non-Hispanic" using `race` *and* `hispan`. But for the moment, just identify "White" respondents using the value in `race`.

Store your results in `results` under the keys `"ex8_avg_income_black"`, `"ex8_avg_income_white"`, and the percentage difference as `ex8_racial_difference`. Please note the wording above when calculating the percentage difference to ensure you get the reference category correct, and interpret your result as well.

In [11]:
results["ex8_avg_income_black"] = df2[df2["race"] == "black/african american/negro"][
    "inctot"
].mean()
results["ex8_avg_income_white"] = df2[df2["race"] == "white"]["inctot"].mean()

results["ex8_racial_difference"] = (
    (results["ex8_avg_income_white"] - results["ex8_avg_income_black"])
    / results["ex8_avg_income_black"]
) * 100

results["ex8_racial_difference"]

print(
    "On an average white Americans make {:.2f}% more compared to Black Americans".format(
        results["ex8_racial_difference"]
    )
)

On an average white Americans make 44.85% more compared to Black Americans


### Exercise 9


As noted above, these estimates are not actually *quite* correct because we aren't using survey weights. To calculate a weighted average that takes into account survey weights, you need to use the following formula:

$$weighted\_mean\_of\_x = \frac{\sum_i x_i * weight_i}{\sum_i weight_i}$$

(As you can see, when $weight_i$ is constant for all observations, this just simplifies to our normal formula for mean values. It is only when weights vary across individuals that weights must be explicitly addressed).

In this data, weights are stored in the variable `perwt`, which is the number of people for which each observation is a stand-in (the inverse of that observations sampling probability). 

Using the formula, re-calculate the *weighted* average income for both populations and store them as `ex9_avg_income_white` and `ex9_avg_income_black`.


In [12]:
whitedf = df2.loc[df2["race"] == "white", ["inctot", "perwt", "hispan"]]
blackdf = df2.loc[df2["race"] == "black/african american/negro", ["inctot", "perwt"]]

results["ex9_avg_income_black"] = sum(blackdf["inctot"] * blackdf["perwt"]) / sum(
    blackdf["perwt"]
)

results["ex9_avg_income_white"] = sum(whitedf["inctot"] * whitedf["perwt"]) / sum(
    whitedf["perwt"]
)

print(
    "The weighted average income of White Americans is ${:.2f}\nand that of Black Americans is ${:.2f}".format(
        results["ex9_avg_income_white"], results["ex9_avg_income_black"]
    )
)

The weighted average income of White Americans is $58361.48
and that of Black Americans is $40430.95


### Exercise 10

Now calculate the weighted average income gap between *non-Hispanic* White Americans and Black Americans. What percentage more do employed White non-Hispanic Americans earn than employed Black Americans? Store as `"ex10_wage_gap"`.

In [13]:
df_nh = whitedf.loc[whitedf["hispan"] == "not hispanic", :]
wt_white_nh = sum(df_nh["inctot"] * df_nh["perwt"]) / sum(df_nh["perwt"])

print(
    "The weighted average income of non-hispanic white americans in the USA is ${:.2f}".format(
        wt_white_nh
    )
)
results["ex10_wage_gap"] = (
    (wt_white_nh - results["ex9_avg_income_black"]) / results["ex9_avg_income_black"]
) * 100

print(
    "After taking the weighted average income, white non-hispanic Americans make {:.2f}% more compared to Black Americans".format(
        results["ex10_wage_gap"]
    )
)

The weighted average income of non-hispanic white americans in the USA is $61669.29
After taking the weighted average income, white non-hispanic Americans make 52.53% more compared to Black Americans


### Exercise 11

Is that greater or less than the difference you found in Exercise 8? Why do you think that's the case?

## Observations
The wage gap between White and Black populations was higher after we subsetted for "not hispanic" whites than the one where we didn't account for the ethnicity in our weighted average income. This might be because the typical understanding of "white" people is different from what the US Census has classified them as, and as a result some of the hispanic populations who might generally be viewed as marginalized communities get inlcuded in the "white" category thereby bringing down the mean values.

In [14]:
# Asserting Keys
assert set(results.keys()) == {
    "ex2_avg_income",
    "ex3_share_making_9999999",
    "ex3_share_making_zero",
    "ex5_avg_income",
    "ex8_avg_income_black",
    "ex8_avg_income_white",
    "ex8_racial_difference",
    "ex9_avg_income_black",
    "ex9_avg_income_white",
    "ex10_wage_gap",
}

In [15]:
for i in results.keys():
    print("{}:{}".format(i, results[i]))

ex2_avg_income:1723646.2703978634
ex3_share_making_9999999:0.1689665333350052
ex3_share_making_zero:0.10557547867738336
ex5_avg_income:40890.177564946454
ex8_avg_income_black:41747.949905123336
ex8_avg_income_white:60473.15372747098
ex8_racial_difference:44.85299006275197
ex9_avg_income_black:40430.953355310274
ex9_avg_income_white:58361.48196061399
ex10_wage_gap:52.52989147705372
